# Why is Loess Fit Lower than Mean Value (and Linear Regression Fit)?



In [1]:
import pandas as pd
import altair as alt
alt.data_transformers.enable('data_server')

# Load panel data. Monthly arrest rate (part 2 crimes per 1,000 people) 
# data for number of localities. 

panel = pd.read_csv(
    "https://github.com/nickeubank/im_baffled/raw/main/arrest_rates.csv.zip"
)
panel.sample(5)

,Unnamed: 0,years_w_decimals,arrest_rate
13834,52174,2017.750000,0.718339
5760,21264,2017.250000,0.799634
2559,9555,2016.083333,1.259675
22474,83962,2016.750000,1.444956
7790,29378,2018.916667,1.021180


In [2]:
# Sample average is 1.41.
panel.arrest_rate.describe()

count    31818.000000
mean         1.413783
std          1.107998
min          0.000000
25%          0.745517
50%          1.137404
75%          1.697478
max         13.695871
Name: arrest_rate, dtype: float64

In [3]:
# And if I do averages for each month, I get 
# a relatively smooth downward trend. 

grouped_means = panel.groupby("years_w_decimals", as_index=False)[["arrest_rate"]].mean()

chart_grouped = (
    alt.Chart(grouped_means)
    .mark_circle(opacity=0.5)
    .encode(
        x=alt.X("years_w_decimals", scale=alt.Scale(zero=False)),
        y=alt.Y("arrest_rate", scale=alt.Scale(zero=False)),
    )
)
chart_grouped

alt.Chart(...)

In [4]:
# A linear regression fits through the monthly averages
# as expected:
reg = (
    alt.Chart(panel)
    .encode(
        x=alt.X("years_w_decimals", scale=alt.Scale(zero=False)),
        y=alt.Y("arrest_rate", scale=alt.Scale(zero=False)),
    )
    .transform_regression(
        "years_w_decimals",
        "arrest_rate",
        method="poly",
        order=1,
    )
    .mark_line()
)
reg + chart_grouped

alt.LayerChart(...)

In [5]:
# But not loess...

loess = (
    alt.Chart(panel)
    .encode(
        x=alt.X("years_w_decimals", scale=alt.Scale(zero=False)),
        y=alt.Y("arrest_rate", scale=alt.Scale(zero=True)),
    )
    .transform_loess(
        on="years_w_decimals",
        loess="arrest_rate",
        bandwidth=0.3
    )
    .mark_line()
)
reg + chart_grouped + loess

alt.LayerChart(...)